In [1]:
import os
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla
from scipy.io import mmread, mmwrite
import torch
from scipy.sparse import csr_matrix, csc_matrix
from gflownet.utils import resize_sparse_tensor, log_memory_usage, malloc_usage, calculate_reward, update_edges_and_convert_to_sparse, trajectory_balance_loss, decompose_ilu_and_create_linear_operator
from typing import List
from gflownet.dataset import *
import gc
from tqdm import tqdm
from gflownet.validate import solve_with_gmres, load_mtx_file, load_vector_mtx, load_mtx_file_csc
from gflownet.utils import torch_sparse_to_csr,  decompose_ilu_and_create_linear_operator, apply_ilu_preconditioner, custom_solve_with_modified_LU
from scipy.sparse import csr_matrix, triu, tril
from scipy.sparse.linalg import gmres, LinearOperator, spilu, splu

In [2]:
path = '/Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/small_ILU/matrices/'

filename = path + 'pores_1.mtx'

A = load_mtx_file_csc(filename)

In [3]:
vector_file = '/Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/small_ILU/b_vectors/pores_1_b.mtx'
b = load_vector_mtx(vector_file)

In [4]:
import scipy.sparse as sp
import scipy.sparse.linalg as spla
from scipy.sparse.linalg import gmres
import tracemalloc  # For tracking memory usage
#Sparse ILU to create baseline preconditioner

# Compute the ILU factorization
ilu = spla.spilu(A)

# Define a function to apply the ILU preconditioner
M_x = lambda x: ilu.solve(x)

# Create a LinearOperator object from the ILU solver function
M = spla.LinearOperator(A.shape, M_x)

In [5]:
print(type(ilu))
print(type(ilu.L))
print(type(ilu.U))
print(M_x)

<class 'SuperLU'>
<class 'scipy.sparse._csc.csc_matrix'>
<class 'scipy.sparse._csc.csc_matrix'>
<function <lambda> at 0x111300940>


In [6]:
def save_ilu_matrix(input_mtx_file):
    # Load the sparse matrix from the .mtx file
    A = mmread(input_mtx_file).tocsc()  # Ensure the matrix is in CSC format
    
    # Perform ILU factorization
    ilu = spla.spilu(A)

    perm_r = ilu.perm_r
    perm_c = ilu.perm_c

    L_mid = ilu.L - csc_matrix(np.eye(A.shape[0]))
    
    whole_ilu_matrix = L_mid + ilu.U



    # Construct output filename and save the LU matrix as an .mtx file
    #original_filename = os.path.splitext(os.path.basename(input_mtx_file))[0]
    #output_file = os.path.join(output_folder, f"{original_filename}_ilu.mtx")
    
    # Save the LU matrix to the specified output folder
    #mmwrite(output_file, ilu_matrix)
    #print(f"ILU matrix saved to {output_file}")

    return whole_ilu_matrix, perm_r, perm_c



In [7]:
leading_path = '/Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/small_ILU/matrices/pores_1.mtx'

whole_ilu_matrix, orig_perm_r, orig_perm_c = save_ilu_matrix(leading_path)

In [8]:

# Get the row and column permutation matrices
perm_r = ilu.perm_r  # Permutation for rows
perm_c = ilu.perm_c  # Permutation for columns


In [9]:
print(perm_r)

[26 23 27 24 29 15 16  9 10  4 25 21 28 18 17 12 11  6  5  2 22 20 19 14
 13  8  7  1  3  0]


In [10]:
orig_x, orig_residuals, orig_num_iterations, orig_elapsed_time = solve_with_gmres(A, b)
print(orig_num_iterations)
print(orig_elapsed_time)

GMRES converged successfully.
18
0.005646944046020508


In [11]:
pre_x, pre_residuals, pre_num_iterations, pre_elapsed_time = solve_with_gmres(A, b, M=M)
print(pre_num_iterations)
print(pre_elapsed_time)

GMRES converged successfully.
4
0.011781930923461914


In [12]:
def decompose_ilu(ilu_matrix):
    # Extract the lower triangular part (including the diagonal) as L
    L = tril(ilu_matrix, k=-1) + csc_matrix(np.eye(ilu_matrix.shape[0]))


    # Extract the upper triangular part (including the diagonal), but we will subtract the identity matrix later
    U = triu(ilu_matrix, format='csc')

    # Now subtract the identity matrix from U to get the strictly upper triangular part



    return L, U

In [13]:
ilu_filename = '/Users/tonylizza/Documents/Machine_Learning/Thesis_Coding/gflownet-spai/data/small_ILU/ilu_matrices/pores_1_ilu.mtx'

ilu_full = load_mtx_file(ilu_filename)

new_M = decompose_ilu_and_create_linear_operator(ilu_full)

/Users/tonylizza/opt/anaconda3/envs/ML2/lib/python3.9/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:437: SparseEfficiencyWarning: spilu converted its input to CSC format
  warn('spilu converted its input to CSC format',


In [14]:
# Compute the ILU factorization
ilu = spilu(A)

# Define a function to apply the ILU preconditioner

L = ilu.L.copy()
U = ilu.U.copy()
perm_c = ilu.perm_c
perm_r = ilu.perm_r


# Create a LinearOperator object from the ILU solver function
M_3 = LinearOperator(A.shape, lambda x: custom_solve_with_modified_LU(x, L, U, perm_r=perm_r, perm_c=perm_c))

In [15]:
print(ilu.perm_r)
print(ilu.perm_c)

[26 23 27 24 29 15 16  9 10  4 25 21 28 18 17 12 11  6  5  2 22 20 19 14
 13  8  7  1  3  0]
[23 24 27 28 29 15 16  9 10  4 25 21 26 18 17 12 11  6  5  3 22 20 19 14
 13  8  7  1  2  0]


In [16]:
print(np.argsort(perm_c))

[29 27 28 19  9 18 17 26 25  7  8 16 15 24 23  5  6 14 13 22 21 11 20  0
  1 10 12  2  3  4]


In [17]:
# Define a function to apply the ILU preconditioner
M_4_x = lambda x: ilu.solve(x)

# Create a LinearOperator object from the ILU solver function
M_4 = spla.LinearOperator(A.shape, M_4_x)

In [18]:
M4_x, M4_residuals, M4_num_iterations, M4_elapsed_time = solve_with_gmres(A, b, M=M_4)
print(M4_num_iterations)
print(M4_elapsed_time)

GMRES converged successfully.
4
0.0063631534576416016


In [19]:
M3_x, M3_residuals, M3_num_iterations, M3_elapsed_time = solve_with_gmres(A, b, M=M_3)
print(M3_num_iterations)
print(M3_elapsed_time)

GMRES converged successfully.
4
0.036392927169799805


In [20]:
test_x = np.random.rand(A.shape[0]).astype(np.float64)

L = L.astype(np.float64)
U = U.astype(np.float64)


# Result from ilu.solve
M_4_x_result = ilu.solve(test_x)

# Result from custom solver
custom_result = custom_solve_with_modified_LU(test_x, L, U, perm_r=perm_r, perm_c=perm_c)

print(np.allclose(M_4_x_result, custom_result))  # Should print True if identical

print(M_4_x_result)
print(custom_result)


True
[-3.04714537e-02  2.61410242e-05 -2.48505622e-02  1.45183044e-05
 -2.20457159e-02  4.62352645e-05 -4.52102068e-02  1.46294437e-01
 -2.17602188e-02  2.68010973e-04 -3.04468668e-02  4.45184985e-04
 -2.49173149e-02  1.58525075e-01 -2.19171271e-02  2.28882669e-02
 -4.48345272e-02  1.76847064e-02 -2.13291218e-02  4.92352783e-05
 -2.36737322e-02 -2.48551103e-01 -2.22853145e-02 -7.15356852e-02
 -2.09513885e-02 -3.58027338e-02 -4.35280234e-02 -1.06494065e-02
 -1.99212928e-02 -6.36457580e-05]
[-3.04714537e-02  2.61410242e-05 -2.48505622e-02  1.45183044e-05
 -2.20457159e-02  4.62352645e-05 -4.52102068e-02  1.46294437e-01
 -2.17602188e-02  2.68010973e-04 -3.04468668e-02  4.45184985e-04
 -2.49173149e-02  1.58525075e-01 -2.19171271e-02  2.28882669e-02
 -4.48345272e-02  1.76847064e-02 -2.13291218e-02  4.92352783e-05
 -2.36737322e-02 -2.48551103e-01 -2.22853145e-02 -7.15356852e-02
 -2.09513885e-02 -3.58027338e-02 -4.35280234e-02 -1.06494065e-02
 -1.99212928e-02 -6.36457580e-05]


In [21]:
LU = spilu(A)
b = np.array(range(30))
Pr = sp.csc_matrix((np.ones(30), (LU.perm_r, np.arange(30))))
Pc = sp.csc_matrix((np.ones(30), (np.arange(30), LU.perm_c)))
x = b.copy()
y = spla.spsolve(LU.L,Pr @ b)
z = spla.spsolve(LU.U, y)
z = Pc @ z
np.linalg.norm((Pc @ z) - spla.spsolve(A,b))

10.717842682668909